In [4]:
# pip install IMDbPY

  Using cached IMDbPY-2022.7.9-py3-none-any.whl (1.2 kB)
  Using cached cinemagoer-2022.12.27-py3-none-any.whl (297 kB)
     |████████████████████████████████| 2.0 MB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 4.7 MB 51.6 MB/s eta 0:00:01
     |████████████████████████████████| 241 kB 18.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

In [5]:
m_df = pd.read_csv('../data/movies_metacritic2.csv')
movie_names = m_df['movie']
movie_names

0                      Roma
1             Amazing Grace
2               Shoplifters
3       Shoah: Four Sisters
4                   Gavagai
               ...         
2788         Big Gold Brick
2789      Mother Schmuckers
2790             Blacklight
2791                Me Time
2792          Asking for It
Name: movie, Length: 2793, dtype: object

In [4]:
names_one = movie_names[:500]
names_one

0                            Roma
1                   Amazing Grace
2                     Shoplifters
3             Shoah: Four Sisters
4                         Gavagai
                  ...            
495                   They Remain
496             7 Days in Entebbe
497                      Souvenir
498                  The Predator
499    Backstabbing for Beginners
Name: movie, Length: 500, dtype: object

In [41]:
from imdb import IMDb

# create an instance of the IMDb class
imdb = IMDb()

# loop through the movie names and get the movie IDs
movies = []
for movie_name in names_one:
    movie_search = imdb.search_movie(movie_name)
    movie_id = movie_search[0].getID()
    movie_title = movie_search[0]["title"]
    movies.append({"id": movie_id, "name": movie_name})


In [42]:
len(movies)

500

In [43]:
# 500 - 1000
names_two = movie_names[500:1000]
names_two

# loop through the movie names and get the movie IDs
movie_ids_two = []
for movie_name in names_two:
    # search for the movie by name
    movie_search = imdb.search_movie(movie_name)
    movie_id = movie_search[0].getID()
    movie_title = movie_search[0]["title"]
    movies.append({"id": movie_id, "name": movie_name})

In [45]:
len(movies)

1000

In [46]:
# 1000 - 1500
names_three = movie_names[1000:1500]
names_three

# loop through the movie names and get the movie IDs
for movie_name in names_three:
    movie_search = imdb.search_movie(movie_name)
    movie_id = movie_search[0].getID()
    movie_title = movie_search[0]["title"]
    movies.append({"id": movie_id, "name": movie_name})

In [47]:
len(movies)

1500

In [50]:
# 1500 - 2000
names_four = movie_names[1500:2000]
names_four

# loop through the movie names and get the movie IDs
for movie_name in names_four:
    try:
        movie_search = imdb.search_movie(movie_name)
        movie_id = movie_search[0].getID()
        movie_title = movie_search[0]["title"]
        movies.append({"id": movie_id, "name": movie_name})
    except IndexError:
        print(f"No search results found for movie: {movie_title}")

No search results found for movie: Castle in the Ground


In [51]:
len(movies)

2092

In [52]:
# 2000 - 2793
names_five = movie_names[2000:2793]
names_five

# loop through the movie names and get the movie IDs
for movie_name in names_five:
    try:
        movie_search = imdb.search_movie(movie_name)
        movie_id = movie_search[0].getID()
        movie_title = movie_search[0]["title"]
        movies.append({"id": movie_id, "name": movie_name})
    except IndexError:
        print(f"No search results found for movie: {movie_title}")

In [85]:
len(movies)

2885

In [99]:
df = pd.DataFrame(movies)
df = df.drop_duplicates(subset=["id"])
df['id'] = df['id'].str.lstrip('tt')
df['id'] = 'tt' + df['id']
df

,id,name
0,tt6155172,Roma
1,tt0454776,Amazing Grace
2,tt8075192,Shoplifters
3,tt6908854,Shoah: Four Sisters
4,tt4515928,Gavagai
...,...,...
2880,tt10308878,Big Gold Brick
2881,tt13648232,Mother Schmuckers
2882,tt14060094,Blacklight
2883,tt14309446,Me Time


In [100]:
df_final = df.copy()
df_final

,id,name
0,tt6155172,Roma
1,tt0454776,Amazing Grace
2,tt8075192,Shoplifters
3,tt6908854,Shoah: Four Sisters
4,tt4515928,Gavagai
...,...,...
2880,tt10308878,Big Gold Brick
2881,tt13648232,Mother Schmuckers
2882,tt14060094,Blacklight
2883,tt14309446,Me Time


In [104]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_reviews(movie_id):
    # make a GET request to the movie's reviews page
    url = f"https://www.imdb.com/title/{movie_id}/reviews"
    response = requests.get(url)

    # parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # find all review containers on the page
    review_containers = soup.find_all('div', class_='lister-item-content')

    # extract the text content, rating, and other information for each review
    reviews = []
    ratings = []
    titles = []
    dates = []
    for container in review_containers:
        review = container.find('div', class_='text').get_text().strip()
        rating = container.find('span', class_='rating-other-user-rating').get_text().strip() if container.find('span', class_='rating-other-user-rating') else None
        title = container.find('a', class_='title').get_text().strip()
        date = container.find('span', class_='review-date').get_text().strip()
        reviews.append(review)
        ratings.append(rating)
        titles.append(title)
        dates.append(date)

    # create a Pandas DataFrame from the review data
    df = pd.DataFrame({'Title': titles, 'Rating': ratings, 'Review': reviews, 'Date': dates})

    return df


In [107]:
reviews_list = []
for movie_id in df_final['id']:
    reviews = get_reviews(movie_id)
    reviews['MovieID'] = movie_id
    reviews_list.append(reviews)

# combine all the DataFrames into a single DataFrame
df = pd.concat(reviews_list, ignore_index=True)

In [108]:
df

,Title,Rating,Review,Date,MovieID
0,"A slice of Life... with a major ""L""...",9/10,The story of Cleo and her 'family' might hit a...,15 December 2018,tt6155172
1,I get it.,8/10,I get why the numerous nominations and I get w...,24 January 2019,tt6155172
2,Already Want to See It Again,9/10,"I already want to see ""Roma"" again.It took me ...",21 December 2018,tt6155172
3,Cuaron's lovely ode to his childhood in Mexico...,None,The title refers to the neighborhood in Mexico...,14 December 2018,tt6155172
4,Stunning cinematography mixed with pathos and ...,5/10,"For the most part, watching Roma was about as ...",18 December 2018,tt6155172
...,...,...,...,...,...
56848,A breath of fresh air,8/10,"Movies like this one and ""Promising Young Woma...",21 February 2022,tt14850172
56849,"Original, relevant, empowering and realistic",9/10,This was non-stop interesting to from start to...,6 March 2022,tt14850172
56850,Excellent Awareness Movie,10/10,The writing and acting are well done. Most peo...,29 May 2022,tt14850172
56851,Ehhhh,3/10,Tried to be something like promising young wom...,16 March 2022,tt14850172


In [109]:
df.describe()

,Title,Rating,Review,Date,MovieID
count,56853,54027,56853,56853,56853
unique,52507,10,56831,4431,2680
top,Disappointing,8/10,It's amazing to watch as things that were at o...,11 December 2020,tt6155172
freq,90,9603,2,75,25


In [113]:
df = df.rename(columns={'MovieID': 'id'})


In [114]:
df_last = pd.merge(df_final, df, on='id', how='inner')

In [115]:
df_last

,id,name,Title,Rating,Review,Date
0,tt6155172,Roma,"A slice of Life... with a major ""L""...",9/10,The story of Cleo and her 'family' might hit a...,15 December 2018
1,tt6155172,Roma,I get it.,8/10,I get why the numerous nominations and I get w...,24 January 2019
2,tt6155172,Roma,Already Want to See It Again,9/10,"I already want to see ""Roma"" again.It took me ...",21 December 2018
3,tt6155172,Roma,Cuaron's lovely ode to his childhood in Mexico...,None,The title refers to the neighborhood in Mexico...,14 December 2018
4,tt6155172,Roma,Stunning cinematography mixed with pathos and ...,5/10,"For the most part, watching Roma was about as ...",18 December 2018
...,...,...,...,...,...,...
56848,tt14850172,Asking for It,A breath of fresh air,8/10,"Movies like this one and ""Promising Young Woma...",21 February 2022
56849,tt14850172,Asking for It,"Original, relevant, empowering and realistic",9/10,This was non-stop interesting to from start to...,6 March 2022
56850,tt14850172,Asking for It,Excellent Awareness Movie,10/10,The writing and acting are well done. Most peo...,29 May 2022
56851,tt14850172,Asking for It,Ehhhh,3/10,Tried to be something like promising young wom...,16 March 2022


In [116]:
df_last.describe()

,id,name,Title,Rating,Review,Date
count,56853,56853,56853,54027,56853,56853
unique,2680,2680,52507,10,56831,4431
top,tt6155172,Roma,Disappointing,8/10,It's amazing to watch as things that were at o...,11 December 2020
freq,25,25,90,9603,2,75


In [117]:
df_last.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56853 entries, 0 to 56852
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      56853 non-null  object
 1   name    56853 non-null  object
 2   Title   56853 non-null  object
 3   Rating  54027 non-null  object
 4   Review  56853 non-null  object
 5   Date    56853 non-null  object
dtypes: object(6)
memory usage: 3.0+ MB


In [118]:
df_last.to_csv('reviews_sample.csv', index=False)

## more pages

In [132]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_reviews(movie_id, num_pages=20):
    # make a GET request to the movie's reviews page
    base_url = f"https://www.imdb.com/title/{movie_id}/reviews"
    reviews = []
    ratings = []
    titles = []
    dates = []

    # scrape each page of reviews
    for page in range(num_pages):
        url = base_url + f"?start={page*10}"
        response = requests.get(url)

        # parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # find all review containers on the page
        review_containers = soup.find_all('div', class_='lister-item-content')

        # extract the text content, rating, and other information for each review
        for container in review_containers:
            review = container.find('div', class_='text').get_text().strip()
            rating = container.find('span', class_='rating-other-user-rating').get_text().strip() if container.find('span', class_='rating-other-user-rating') else None
            title = container.find('a', class_='title').get_text().strip()
            date = container.find('span', class_='review-date').get_text().strip()
            reviews.append(review)
            ratings.append(rating)
            titles.append(title)
            dates.append(date)
    
    df = pd.DataFrame({'Title': titles, 'Rating': ratings, 'Review': reviews, 'Date': dates})

    return df


In [133]:
reviews_list = []
for movie_id in df_final['id'][:50]:
    reviews = get_reviews(movie_id)
    reviews['MovieID'] = movie_id
    reviews_list.append(reviews)

# combine all the DataFrames into a single DataFrame
df2 = pd.concat(reviews_list, ignore_index=True)

In [134]:
df2

,Title,Rating,Review,Date,MovieID
0,"A slice of Life... with a major ""L""...",9/10,The story of Cleo and her 'family' might hit a...,15 December 2018,tt6155172
1,I get it.,8/10,I get why the numerous nominations and I get w...,24 January 2019,tt6155172
2,Already Want to See It Again,9/10,"I already want to see ""Roma"" again.It took me ...",21 December 2018,tt6155172
3,Cuaron's lovely ode to his childhood in Mexico...,None,The title refers to the neighborhood in Mexico...,14 December 2018,tt6155172
4,Stunning cinematography mixed with pathos and ...,5/10,"For the most part, watching Roma was about as ...",18 December 2018,tt6155172
...,...,...,...,...,...
21175,Great experience,9/10,"I loved this movie, it is one of the most auth...",7 January 2019,tt5536610
21176,Story of my life,10/10,It was extremely shocking to me that an actres...,16 October 2018,tt5536610
21177,"Life isn't fair for all, of course !!",9/10,"Phew, What a take on life and What a movie, Di...",31 July 2021,tt5536610
21178,Sloppy storytelling,2/10,"Middle aged, intellectual New Yorkers Richard ...",24 October 2018,tt5536610


In [135]:
df2.describe()

,Title,Rating,Review,Date,MovieID
count,21180,19740,21180,21180,21180
unique,1053,10,1057,658,50
top,Beautiful,10/10,"It's a triumph of filmmaking, powerful and min...",12 November 2018,tt6155172
freq,60,4180,40,180,500


In [137]:
reviews_list = []
for movie_id in df_final['id'][50:150]:
    reviews = get_reviews(movie_id)
    reviews['MovieID'] = movie_id
    reviews_list.append(reviews)

# combine all the DataFrames into a single DataFrame
df3 = pd.concat(reviews_list, ignore_index=True)

In [138]:
df3.describe()

,Title,Rating,Review,Date,MovieID
count,36380,33980,36380,36380,36380
unique,1809,10,1819,1027,95
top,Brilliant,8/10,Amazing true story of a small team of police i...,17 November 2018,tt7349662
freq,60,7340,20,240,500


In [139]:
reviews_list = []
for movie_id in df_final['id'][150:300]:
    reviews = get_reviews(movie_id)
    reviews['MovieID'] = movie_id
    reviews_list.append(reviews)

# combine all the DataFrames into a single DataFrame
df4 = pd.concat(reviews_list, ignore_index=True)

In [140]:
df_combine = pd.concat([df2, df3, df4])
df_combine

,Title,Rating,Review,Date,MovieID
0,"A slice of Life... with a major ""L""...",9/10,The story of Cleo and her 'family' might hit a...,15 December 2018,tt6155172
1,I get it.,8/10,I get why the numerous nominations and I get w...,24 January 2019,tt6155172
2,Already Want to See It Again,9/10,"I already want to see ""Roma"" again.It took me ...",21 December 2018,tt6155172
3,Cuaron's lovely ode to his childhood in Mexico...,None,The title refers to the neighborhood in Mexico...,14 December 2018,tt6155172
4,Stunning cinematography mixed with pathos and ...,5/10,"For the most part, watching Roma was about as ...",18 December 2018,tt6155172
...,...,...,...,...,...
55835,Barely there and mostly boring zombie effort,3/10,"Traveling through the Australian outback, a ma...",8 January 2019,tt3860916
55836,It was better as a short,1/10,The short was very well done. Why? Because it ...,19 May 2018,tt3860916
55837,A decent zombie flick!,7/10,"Cargo is generally enjoyable, but it does mana...",24 May 2018,tt3860916
55838,One of the best storylines for a Z flick top 3...,9/10,Definitely a warm heartfelt zombie flick maybe...,19 May 2018,tt3860916


In [141]:
df_combine.to_csv('reviews_part1.csv', index=False)

In [142]:
reviews_list = []
for movie_id in df_final['id'][300:450]:
    reviews = get_reviews(movie_id)
    reviews['MovieID'] = movie_id
    reviews_list.append(reviews)

# combine all the DataFrames into a single DataFrame
df5 = pd.concat(reviews_list, ignore_index=True)

In [143]:
df5

,Title,Rating,Review,Date,MovieID
0,Dark history piece. Great to look at,8/10,I liked. A very raw irish drama with strong mo...,21 February 2018,tt3208026
1,Loved this film,9/10,There is a lot in this that reminds me of elem...,2 April 2019,tt3208026
2,"Not really the Famine film we were promised, b...",8/10,I'm not going to lie - I fully expected to hat...,19 September 2018,tt3208026
3,Please do not refer to this as English history,10/10,This movie depicts just one atrocity committed...,28 June 2019,tt3208026
4,A moving tale from an inglorious past..,8/10,"A great script, cast, acting, covering a great...",20 January 2020,tt3208026
...,...,...,...,...,...
62955,Red Sparrow occasionally dances elegantly but ...,6/10,"When we talk about the spy genre, most of us w...",9 March 2018,tt2873282
62956,Predictable twaddle.,2/10,Based upon the book of the same name by ex-CIA...,2 March 2018,tt2873282
62957,Under Rated from fan's and critic's Must see Film,9/10,Under rated double agent Spy Thriller . Jennif...,17 March 2018,tt2873282
62958,Which side is she on?,9/10,Enjoyable spy thriller. The violence is not gl...,11 April 2019,tt2873282


In [145]:
reviews_list = []
for movie_id in df_final['id'][450:700]:
    reviews = get_reviews(movie_id)
    reviews['MovieID'] = movie_id
    reviews_list.append(reviews)

# combine all the DataFrames into a single DataFrame
df6 = pd.concat(reviews_list, ignore_index=True)

In [148]:
reviews_list = []
for movie_id in df_final['id'][700:850]:
    reviews = get_reviews(movie_id)
    reviews['MovieID'] = movie_id
    reviews_list.append(reviews)

# combine all the DataFrames into a single DataFrame
df7 = pd.concat(reviews_list, ignore_index=True)

In [150]:
reviews_list = []
for movie_id in df_final['id'][850:1000]:
    reviews = get_reviews(movie_id)
    reviews['MovieID'] = movie_id
    reviews_list.append(reviews)

# combine all the DataFrames into a single DataFrame
df8 = pd.concat(reviews_list, ignore_index=True)

In [152]:
reviews_list = []
for movie_id in df_final['id'][1000:1150]:
    reviews = get_reviews(movie_id)
    reviews['MovieID'] = movie_id
    reviews_list.append(reviews)

# combine all the DataFrames into a single DataFrame
df9 = pd.concat(reviews_list, ignore_index=True)

In [154]:
reviews_list = []
for movie_id in df_final['id'][1150:1300]:
    reviews = get_reviews(movie_id)
    reviews['MovieID'] = movie_id
    reviews_list.append(reviews)

# combine all the DataFrames into a single DataFrame
df10 = pd.concat(reviews_list, ignore_index=True)

In [155]:
df_combine2 = pd.concat([df5, df6, df7, df8, df9, df10])
df_combine2

,Title,Rating,Review,Date,MovieID
0,Dark history piece. Great to look at,8/10,I liked. A very raw irish drama with strong mo...,21 February 2018,tt3208026
1,Loved this film,9/10,There is a lot in this that reminds me of elem...,2 April 2019,tt3208026
2,"Not really the Famine film we were promised, b...",8/10,I'm not going to lie - I fully expected to hat...,19 September 2018,tt3208026
3,Please do not refer to this as English history,10/10,This movie depicts just one atrocity committed...,28 June 2019,tt3208026
4,A moving tale from an inglorious past..,8/10,"A great script, cast, acting, covering a great...",20 January 2020,tt3208026
...,...,...,...,...,...
55415,Accidentally found,10/10,One the most underrated movie of 2020. I sat d...,13 June 2021,tt8980602
55416,Pretty good!,7/10,It's a pretty good movie. Not really any dull ...,17 August 2021,tt8980602
55417,The surprise star of 2020,8/10,Sometimes you stumble across a movie having no...,6 January 2021,tt8980602
55418,brutal mystery wrapped in a light tone,7/10,Abe Applebaum (Adam Brody) is a world-weary 32...,14 July 2022,tt8980602


In [156]:
df_combine2.to_csv('reviews_part2.csv', index=False)

In [160]:
reviews_list = []
for movie_id in df_final['id'][1300:1450]:
    reviews = get_reviews(movie_id)
    reviews['MovieID'] = movie_id
    reviews_list.append(reviews)

# combine all the DataFrames into a single DataFrame
df11 = pd.concat(reviews_list, ignore_index=True)

KeyboardInterrupt: 

In [161]:
df11.describe()

,Title,Rating,Review,Date,MovieID
count,82540,78560,82540,82540,82540
unique,4014,10,4124,1160,196
top,Meh,8/10,Finding the perfect movie when you can't go ou...,14 November 2020,tt11615290
freq,160,15640,40,460,500


In [162]:
reviews_list = []
for movie_id in df_final['id'][1500:1700]:
    reviews = get_reviews(movie_id)
    reviews['MovieID'] = movie_id
    reviews_list.append(reviews)

# combine all the DataFrames into a single DataFrame
df12 = pd.concat(reviews_list, ignore_index=True)

In [165]:
reviews_list = []
for movie_id in df_final['id'][1700:1850]:
    reviews = get_reviews(movie_id)
    reviews['MovieID'] = movie_id
    reviews_list.append(reviews)

# combine all the DataFrames into a single DataFrame
df13 = pd.concat(reviews_list, ignore_index=True)

In [167]:
reviews_list = []
for movie_id in df_final['id'][1850:2000]:
    reviews = get_reviews(movie_id)
    reviews['MovieID'] = movie_id
    reviews_list.append(reviews)

# combine all the DataFrames into a single DataFrame
df14 = pd.concat(reviews_list, ignore_index=True)

In [168]:
df14.describe()

,Title,Rating,Review,Date,MovieID
count,59440,56780,59440,59440,59440
unique,2895,10,2972,890,148
top,Beautiful,8/10,Writer-director Kenneth Branagh tells a semi-a...,6 August 2021,tt12789558
freq,140,10880,20,440,500


In [169]:
reviews_list = []
for movie_id in df_final['id'][2000:2150]:
    reviews = get_reviews(movie_id)
    reviews['MovieID'] = movie_id
    reviews_list.append(reviews)

# combine all the DataFrames into a single DataFrame
df15 = pd.concat(reviews_list, ignore_index=True)

In [180]:
df_combine3 = pd.concat([df11, df12, df13, df14, df15])
df_combine3.to_csv('reviews_part3.csv', index=False)
df_combine3

,Title,Rating,Review,Date,MovieID
0,Yale Yauch...,9/10,Mike and Adam get on stage and tell it like it...,24 April 2020,tt11615290
1,Finally... Thank you for the moment,10/10,This is a quintessentially Beastie Boys moment...,24 April 2020,tt11615290
2,A must see for fans and sorta-fans,10/10,I'm glad I caught this documentary/live show o...,24 April 2020,tt11615290
3,Grown up nostalgia trip.,7/10,"Although this made me feel really old, I liked...",23 July 2020,tt11615290
4,"Influential, engaging and a must-see.",10/10,"""Beastie Boys Story"" is an honest, engaging an...",28 April 2020,tt11615290
...,...,...,...,...,...
67395,"No big surprises here - it's a heist film, min...",5/10,With a fairly predictable plot and one-dimensi...,29 October 2021,tt13024674
67396,To be endured not enjoyed,2/10,At the second safe here we go again with some ...,14 February 2022,tt13024674
67397,too smooth...,5/10,"The plot far too plantastic, or made of plasti...",1 November 2021,tt13024674
67398,Zombie Lite Bank Caper,7/10,Army of Thieves: Zombie Lite prequel to Army o...,1 November 2021,tt13024674


In [171]:
reviews_list = []
for movie_id in df_final['id'][2150:2300]:
    reviews = get_reviews(movie_id)
    reviews['MovieID'] = movie_id
    reviews_list.append(reviews)

# combine all the DataFrames into a single DataFrame
df16 = pd.concat(reviews_list, ignore_index=True)

In [172]:
reviews_list = []
for movie_id in df_final['id'][2300:2450]:
    reviews = get_reviews(movie_id)
    reviews['MovieID'] = movie_id
    reviews_list.append(reviews)

# combine all the DataFrames into a single DataFrame
df17 = pd.concat(reviews_list, ignore_index=True)

In [173]:
df_final['id']

0        tt6155172
1        tt0454776
2        tt8075192
3        tt6908854
4        tt4515928
           ...    
2880    tt10308878
2881    tt13648232
2882    tt14060094
2883    tt14309446
2884    tt14850172
Name: id, Length: 2759, dtype: object

In [174]:
reviews_list = []
for movie_id in df_final['id'][2450:2600]:
    reviews = get_reviews(movie_id)
    reviews['MovieID'] = movie_id
    reviews_list.append(reviews)

# combine all the DataFrames into a single DataFrame
df18 = pd.concat(reviews_list, ignore_index=True)

In [176]:
reviews_list = []
for movie_id in df_final['id'][2600:2760]:
    reviews = get_reviews(movie_id)
    reviews['MovieID'] = movie_id
    reviews_list.append(reviews)

# combine all the DataFrames into a single DataFrame
df19 = pd.concat(reviews_list, ignore_index=True)

In [177]:
df19

,Title,Rating,Review,Date,MovieID
0,"Atmospheric, colorful fun!",10/10,I think the problem with TV sitcoms these days...,1 September 2002,tt0057773
1,The Perfect Family,None,"Oh, I wish they made shows like The Munsters t...",12 August 2002,tt0057773
2,The Munsters!,None,One of the greatest television shows of all ti...,21 February 2005,tt0057773
3,"""Herman Munster don't you dare!""",None,This show can easily be dismissed as being so ...,6 February 2006,tt0057773
4,Lurch who?,None,Although I liked the Addams Family quite a bit...,8 April 2001,tt0057773
...,...,...,...,...,...
74615,A breath of fresh air,8/10,"Movies like this one and ""Promising Young Woma...",21 February 2022,tt14850172
74616,"Original, relevant, empowering and realistic",9/10,This was non-stop interesting to from start to...,6 March 2022,tt14850172
74617,Excellent Awareness Movie,10/10,The writing and acting are well done. Most peo...,29 May 2022,tt14850172
74618,Ehhhh,3/10,Tried to be something like promising young wom...,16 March 2022,tt14850172


In [179]:
df_combine4 = pd.concat([df16, df17, df18, df19])
df_combine4.to_csv('reviews_part4.csv', index=False)
df_combine4

,Title,Rating,Review,Date,MovieID
0,"Bad sequel, I'm done with it.",3/10,I remember I enjoyed watching the first Escape...,12 December 2021,tt9844522
1,Pay strong attention to which version you watch!,6/10,"Note this movie has 2 versions, one is the cin...",13 October 2022,tt9844522
2,Predictable n not at all at par with its prede...,6/10,I enjoyed the first part n it has some cool tr...,8 September 2021,tt9844522
3,"Not As Good As The First, But Worth Watching I...",7/10,If you loved the first one you should watch th...,10 October 2021,tt9844522
4,Shorter and dumber than the first one,2/10,This is a direct sequel to the surprisingly su...,31 August 2021,tt9844522
...,...,...,...,...,...
74615,A breath of fresh air,8/10,"Movies like this one and ""Promising Young Woma...",21 February 2022,tt14850172
74616,"Original, relevant, empowering and realistic",9/10,This was non-stop interesting to from start to...,6 March 2022,tt14850172
74617,Excellent Awareness Movie,10/10,The writing and acting are well done. Most peo...,29 May 2022,tt14850172
74618,Ehhhh,3/10,Tried to be something like promising young wom...,16 March 2022,tt14850172


In [181]:
df_combined = pd.concat([df_combine, df_combine2, df_combine3, df_combine4])

,Title,Rating,Review,Date,MovieID
0,"A slice of Life... with a major ""L""...",9/10,The story of Cleo and her 'family' might hit a...,15 December 2018,tt6155172
1,I get it.,8/10,I get why the numerous nominations and I get w...,24 January 2019,tt6155172
2,Already Want to See It Again,9/10,"I already want to see ""Roma"" again.It took me ...",21 December 2018,tt6155172
3,Cuaron's lovely ode to his childhood in Mexico...,None,The title refers to the neighborhood in Mexico...,14 December 2018,tt6155172
4,Stunning cinematography mixed with pathos and ...,5/10,"For the most part, watching Roma was about as ...",18 December 2018,tt6155172
...,...,...,...,...,...
55835,Barely there and mostly boring zombie effort,3/10,"Traveling through the Australian outback, a ma...",8 January 2019,tt3860916
55836,It was better as a short,1/10,The short was very well done. Why? Because it ...,19 May 2018,tt3860916
55837,A decent zombie flick!,7/10,"Cargo is generally enjoyable, but it does mana...",24 May 2018,tt3860916
55838,One of the best storylines for a Z flick top 3...,9/10,Definitely a warm heartfelt zombie flick maybe...,19 May 2018,tt3860916


In [182]:
df_combined.to_csv('reviews_500.csv', index=False)
df_combined

,Title,Rating,Review,Date,MovieID
0,"A slice of Life... with a major ""L""...",9/10,The story of Cleo and her 'family' might hit a...,15 December 2018,tt6155172
1,I get it.,8/10,I get why the numerous nominations and I get w...,24 January 2019,tt6155172
2,Already Want to See It Again,9/10,"I already want to see ""Roma"" again.It took me ...",21 December 2018,tt6155172
3,Cuaron's lovely ode to his childhood in Mexico...,None,The title refers to the neighborhood in Mexico...,14 December 2018,tt6155172
4,Stunning cinematography mixed with pathos and ...,5/10,"For the most part, watching Roma was about as ...",18 December 2018,tt6155172
...,...,...,...,...,...
74615,A breath of fresh air,8/10,"Movies like this one and ""Promising Young Woma...",21 February 2022,tt14850172
74616,"Original, relevant, empowering and realistic",9/10,This was non-stop interesting to from start to...,6 March 2022,tt14850172
74617,Excellent Awareness Movie,10/10,The writing and acting are well done. Most peo...,29 May 2022,tt14850172
74618,Ehhhh,3/10,Tried to be something like promising young wom...,16 March 2022,tt14850172
